In [ ]:
import pandas as pd

url_donnees_depistage = "https://www.data.gouv.fr/fr/datasets/r/426bab53-e3f5-4c6a-9d54-dba4442b3dbc"
df_depistage = pd.read_csv(url_donnees_depistage, sep=";")

#on se débarrasse des colonnes inutiles et qu'on doit recalculer
df_depistage = df_depistage.drop(columns=['cl_age90',"Ti","Tp","Td"])
df_depistage = df_depistage.rename(columns = {'pop':'population','P':'nb patients positifs',
'T':'nb patients testes'})

#conversion des valeurs afin de pouvoir faire des calculs
df_depistage["population"] = df_depistage["population"].str.replace(',','.').astype(float)
df_depistage["nb patients positifs"] = df_depistage["nb patients positifs"].str.replace(',','.').astype(float)
df_depistage["nb patients testes"] = df_depistage["nb patients testes"].str.replace(',','.').astype(float)

df_depistage["dep"] = df_depistage['dep'].astype(str)
df_depistage["mois"] = df_depistage["jour"].astype(str).str[5:7]
df_depistage["annee"] = df_depistage["jour"].astype(str).str[:4]

df_depistage["id"] = df_depistage["dep"] + "_" + df_depistage["annee"] + "_" + df_depistage["mois"]


In [ ]:
df_depistage = df_depistage.groupby("id").agg({"dep":"first","mois":"first","annee":"first","population":"mean","nb patients positifs":"sum",
"nb patients testes":"sum"})

#définition de taux en pourcentage
df_depistage['taux de positivite'] = df_depistage['nb patients positifs']*100/df_depistage['nb patients testes']
df_depistage['taux de depistage'] = df_depistage['nb patients testes']*100/df_depistage['population']
df_depistage["taux d'incidence"] = df_depistage['nb patients positifs']*100/df_depistage['population']

df_depistage.head(10)

In [ ]:
df_test = df_depistage.set_index("dep")
df_test.head(10)

In [2]:
import pandas as pd
url = "https://www.data.gouv.fr/fr/datasets/r/eceb9fb4-3ebc-4da3-828d-f5939712600a"
df_urgences = pd.read_csv(url, sep=";")

df_urgences = df_urgences[['dep','date_de_passage','sursaud_cl_age_corona','nbre_pass_corona', 'nbre_pass_tot','nbre_hospit_corona','nbre_acte_corona','nbre_acte_tot']]
#on remplit les valeurs non renseignées par 0 
df_urgences.fillna(0)
df_urgences["dep"] = df_urgences['dep'].astype(str)
df_urgences["mois"] = df_urgences["date_de_passage"].astype(str).str[5:7]
df_urgences["annee"] = df_urgences["date_de_passage"].astype(str).str[:4]

df_urgences["id"] = df_urgences["dep"] + "_" + df_urgences["annee"] + "_" + df_urgences["mois"] + "_" + df_urgences["sursaud_cl_age_corona"].astype(str)

df_urgences = df_urgences.groupby("id").agg({"dep":"first","mois":"first","annee":"first",
"sursaud_cl_age_corona":"first","nbre_pass_corona":"sum","nbre_pass_tot":"sum",
"nbre_hospit_corona":"sum","nbre_acte_corona":"sum","nbre_acte_tot":"sum"})

df_urgences.head(10)

/tmp/ipykernel_384206/2335086498.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_urgences = pd.read_csv(url, sep=";")


,dep,mois,annee,sursaud_cl_age_corona,nbre_pass_corona,nbre_pass_tot,nbre_hospit_corona,nbre_acte_corona,nbre_acte_tot
id,,,,,,,,,
10_2022_12_0,10,12,2022,0,13.0,1238.0,8.0,8.0,899.0
10_2022_12_1,10,12,2022,1,2.0,204.0,0.0,1.0,158.0
10_2022_12_2,10,12,2022,2,0.0,92.0,0.0,0.0,83.0
10_2022_12_3,10,12,2022,3,1.0,404.0,0.0,4.0,462.0
10_2022_12_4,10,12,2022,4,2.0,215.0,1.0,2.0,110.0
10_2022_12_5,10,12,2022,5,2.0,98.0,1.0,1.0,31.0
10_2022_12_6,10,12,2022,6,6.0,225.0,6.0,0.0,55.0
10_2023_01_0,10,01,2023,0,9.0,5426.0,5.0,7.0,4227.0
10_2023_01_1,10,01,2023,1,1.0,653.0,0.0,0.0,663.0


In [ ]:
import pandas as pd
url_bis = "https://www.data.gouv.fr/fr/datasets/r/6fadff46-9efd-4c53-942a-54aca783c30c"
df_hosp = pd.read_csv(url_bis, sep = ";")

df_hosp = df_hosp.rename(columns = {'incid_hosp':'nb hospitalisations','incid_rea':'nb reanimations',
'incid_dc':'nb deces','incid_rad':'nb retour au domicile'})
df_hosp["dep"] = df_hosp["dep"].astype(str)
df_hosp["mois"] = df_hosp["jour"].astype(str).str[5:7]
df_hosp["annee"] = df_hosp["jour"].astype(str).str[:4]

df_hosp["id"] = df_hosp["dep"] + "_" + df_hosp["annee"] + "_" + df_hosp["mois"] 
df_hosp = df_hosp.groupby("id").agg({"dep":"first","mois":"first","annee":"first","nb hospitalisations":"sum","nb reanimations":"sum",
 "nb deces":"sum", "nb retour au domicile":"sum"})

df_hosp.head(10)


In [ ]:
!pip install openpyxl

In [2]:
import pandas as pd
from fileopener import *

# import des dataframe grâce au fileopener
file = "donnees.txt"
df_list = dataframe_creator(file)
# ordre des df : hospitalisations, urgences, dépistages, etablissements, espérance de vie, niveaux de vie, pauvreté monétaire, vieillissement, population selon age
df_hosp = df_list[0]
df_urgences = df_list[1]
df_depistage = df_list[2]
df_etab = df_list[3]
df_espvie = df_list[4]
df_nvvie = df_list[5]
df_pauv = df_list[6]
df_vieil = df_list[7]
df_pop = df_list[8]
df_deces = df_list[9]


In [ ]:
df_hosp = df_hosp.rename(columns = {'incid_hosp':'nb hospitalisations','incid_rea':'nb reanimations',
'incid_dc':'nb deces','incid_rad':'nb retour au domicile'})
df_hosp["dep"] = df_hosp["dep"].astype(str)
df_hosp["mois"] = df_hosp["jour"].astype(str).str[5:7]
df_hosp["annee"] = df_hosp["jour"].astype(str).str[:4]

df_hosp["id"] = df_hosp["dep"] + "_" + df_hosp["annee"] + "_" + df_hosp["mois"] 
df_hosp = df_hosp.groupby("id").agg({"dep":"first","mois":"first","annee":"first","nb hospitalisations":"sum","nb reanimations":"sum",
 "nb deces":"sum", "nb retour au domicile":"sum"})

"""
#on remet les départements comme index de la df
df_hosp = df_hosp.set_index("dep")
"""


In [ ]:
#on retire les territoires d'outre-mer pour garder les départements
tom_list = ['977','978','986','987','988','984','989']
df_hosp = df_hosp.drop(df_hosp[df_hosp["dep"].isin(tom_list)].index)
display(df_hosp.head())

In [ ]:
dep_list = pd.unique(df_hosp["dep"])
column_hosp = df_hosp.columns.to_list()
index_list = []

for dep in dep_list:
    for annee in range(2019,2024):
        for mois in range(1,13):
            id = f'{dep}_{annee}_{mois:02d}'
            index_list.append(id)

df_hosp_fill = pd.DataFrame(0, index=index_list, columns=column_hosp)


In [ ]:
print(dep_list)

In [ ]:
display(df_hosp.head())
display(df_hosp_fill.head())

In [ ]:

df_hosp_test = pd.concat([df_hosp, df_hosp_fill], ignore_index=False)




In [ ]:
df_hosp_test = df_hosp_test.reset_index().drop_duplicates(subset="index", keep="first").set_index("index").sort_index()
display(df_hosp_test.head())

In [ ]:
print(df_hosp_test.nunique())
print(len(df_hosp_test))

In [2]:
# Nettoyage de la base des urgences

df_urgences = df_urgences[['dep','date_de_passage','sursaud_cl_age_corona','nbre_pass_corona', 'nbre_pass_tot',
 "nbre_hospit_corona","nbre_acte_corona","nbre_acte_tot"]]
#on remplit les valeurs non renseignées par 0
df_urgences.fillna(0)
df_urgences["dep"] = df_urgences['dep'].astype(str)
df_urgences["mois"] = df_urgences["date_de_passage"].astype(str).str[5:7]
df_urgences["annee"] = df_urgences["date_de_passage"].astype(str).str[:4]

df_urgences["id"] = df_urgences["dep"] + "_" + df_urgences["annee"] + "_" + df_urgences["mois"] + "_" + df_urgences["sursaud_cl_age_corona"].astype(str)
df_urgences["id_bis"] = df_urgences["dep"] + "_" + df_urgences["annee"] + "_" + df_urgences["mois"]

df_urgences = df_urgences.groupby("id").agg({"id_bis":"first","dep":"first","mois":"first","annee":"first",
"sursaud_cl_age_corona":"first","nbre_pass_corona":"sum","nbre_pass_tot":"sum",
"nbre_hospit_corona":"sum","nbre_acte_corona":"sum","nbre_acte_tot":"sum"})


#on remet les départements comme index de la df
df_urgences = df_urgences.set_index("id_bis")

In [3]:
df_urgences_pivoted = df_urgences.pivot_table(
    index=df_urgences.index,
    columns="sursaud_cl_age_corona",
    values=['nbre_pass_corona','nbre_pass_tot','nbre_acte_corona','nbre_acte_tot']
)

df_urgences_pivoted.columns = [f"{col[0]}_age_{col[1]}" for col in df_urgences_pivoted.columns]

df_urgences_pivoted.head(20)

,nbre_acte_corona_age_0,nbre_acte_corona_age_1,nbre_acte_corona_age_2,nbre_acte_corona_age_3,nbre_acte_corona_age_4,nbre_acte_corona_age_5,nbre_acte_corona_age_6,nbre_acte_tot_age_0,nbre_acte_tot_age_1,nbre_acte_tot_age_2,...,nbre_pass_corona_age_4,nbre_pass_corona_age_5,nbre_pass_corona_age_6,nbre_pass_tot_age_0,nbre_pass_tot_age_1,nbre_pass_tot_age_2,nbre_pass_tot_age_3,nbre_pass_tot_age_4,nbre_pass_tot_age_5,nbre_pass_tot_age_6
id_bis,,,,,,,,,,,,,,,,,,,,,
01_2022_12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,7.0,16.0,2372.0,264.0,173.0,818.0,477.0,227.0,412.0
01_2023_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,7.0,19.0,10822.0,757.0,1311.0,4102.0,2102.0,912.0,1625.0
01_2023_02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,2.0,4.0,10353.0,721.0,1185.0,4044.0,2002.0,902.0,1482.0
01_2023_03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,6.0,28.0,12351.0,848.0,1791.0,4667.0,2263.0,991.0,1777.0
01_2023_04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18.0,16.0,45.0,11084.0,821.0,1234.0,4176.0,2271.0,942.0,1623.0
01_2023_05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,11.0,16.0,11704.0,769.0,1673.0,4354.0,2330.0,981.0,1583.0
01_2023_06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,2.0,11457.0,726.0,1592.0,4366.0,2282.0,920.0,1554.0
01_2023_07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,5.0,10477.0,627.0,998.0,4024.0,2288.0,952.0,1563.0
01_2023_08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,9.0,12.0,10165.0,579.0,856.0,3845.0,2235.0,984.0,1650.0


In [4]:
df_depistage = df_depistage.drop(columns=['cl_age90',"Ti","Tp","Td"])
df_depistage = df_depistage.rename(columns = {'pop':'population','P':'nb patients positifs',
'T':'nb patients testes'})

#conversion des valeurs afin de pouvoir faire des calculs
df_depistage["population"] = df_depistage["population"].str.replace(',','.').astype(float)
df_depistage["nb patients positifs"] = df_depistage["nb patients positifs"].str.replace(',','.').astype(float)
df_depistage["nb patients testes"] = df_depistage["nb patients testes"].str.replace(',','.').astype(float)

df_depistage["dep"] = df_depistage['dep'].astype(str)
df_depistage["mois"] = df_depistage["jour"].astype(str).str[5:7]
df_depistage["annee"] = df_depistage["jour"].astype(str).str[:4]

#création d'une id pour pouvoir regrouper par département/mois/année
df_depistage["id"] = df_depistage["dep"] + "_" + df_depistage["annee"] + "_" + df_depistage["mois"]

df_depistage = df_depistage.groupby("id").agg({"dep":"first","mois":"first","annee":"first","population":"mean","nb patients positifs":"sum",
"nb patients testes":"sum"})

#définition de taux en pourcentage
df_depistage['taux de positivite'] = df_depistage['nb patients positifs']*100/df_depistage['nb patients testes']
df_depistage['taux de depistage'] = df_depistage['nb patients testes']*100/df_depistage['population']
df_depistage["taux d'incidence"] = df_depistage['nb patients positifs']*100/df_depistage['population']

display(df_depistage.head(10))

,dep,mois,annee,population,nb patients positifs,nb patients testes,taux de positivite,taux de depistage,taux d'incidence
id,,,,,,,,,
01_2020_05,01,05,2020,656955.0,83.0,5986.0,1.386569,0.911174,0.012634
01_2020_06,01,06,2020,656955.0,102.0,8499.0,1.200141,1.293696,0.015526
01_2020_07,01,07,2020,656955.0,150.0,15984.0,0.938438,2.433043,0.022833
01_2020_08,01,08,2020,656955.0,765.0,23369.0,3.273568,3.557169,0.116446
01_2020_09,01,09,2020,656955.0,2348.0,43071.0,5.451464,6.556157,0.357407
01_2020_10,01,10,2020,656955.0,14920.0,75977.0,19.637522,11.565023,2.271084
01_2020_11,01,11,2020,656955.0,11919.0,69038.0,17.264405,10.508787,1.814280
01_2020_12,01,12,2020,656955.0,5010.0,91603.0,5.469253,13.943573,0.762609
01_2021_01,01,01,2021,656955.0,5817.0,63391.0,9.176382,9.649215,0.885449


In [ ]:
# Nettoyage de la base des établissements

"""on fait en sorte d'avoir des noms pour chaque colonnes pour pouvoir travailler sur le dataFrame"""
names_columns = ["finess", "etalab"]+[str(x) for x in range(30)]

df_etab.columns = names_columns

"""On sélectionne les colonnes intéressantes """
df_etab = df_etab.loc[:,["11","12","17","23",]]

df_etab.rename(columns={"11" : "num_dep", "12" : "nom_dep", "17" : "type_centre", "23" : "statut_centre"}, inplace=True)

"""On souhaite sélectionner les lieux pouvant avoir un lien avec la guérison/dépistage du covid uniquement"""
word_covid = ["hospi", "pharma", "maison de santé", "Laboratoire d'Analyses"]
df_etab["etat_soin_covid"] = df_etab["type_centre"].str.contains(pat="|".join(word_covid), case=False, na=False).astype(int)

df_etab.loc[df_etab["etat_soin_covid"]==1]["type_centre"].unique()


"""Certaines cathégories d'établissement se sont glissée dans notre large sélection, on les retire donc"""
line_to_drop = (["Centre Hospitalier Spécialisé lutte Maladies Mentales", "Maison de Santé pour Maladies Mentales",
"Laboratoire pharmaceutique préparant délivrant allergènes", "Syndicat Inter Hospitalier (S.I.H.)"]
    )
df_etab.loc[df_etab["type_centre"].isin(line_to_drop),"etat_soin_covid"] = 0

df_etab = df_etab.groupby(["nom_dep", "num_dep"])["etat_soin_covid"].sum().reset_index()
df_etab.sort_values(by = "etat_soin_covid",ascending=False)

display(df_etab)

In [7]:
# Nettoyage de la base de l'espérance de vie

df_espvie = df_espvie.rename(
    columns= {"Unnamed: 0": "num_dep", "Unnamed: 1" : "nom_dep", "Hommes" : "esp_de_vie_homme_naissance",
    "Femmes" : "esp_de_vie_femme_naissance",
    "Hommes.1" : "esp_de_vie_homme_a_60",
    "Femmes.1": "esp_de_vie_femme_a_60",
    "Hommes.2" : "esp_de_vie_homme_a_65",
    "Femmes.2" : "esp_de_vie_femme_a_65"}
    )
for col in df_espvie.select_dtypes(["int64", "float64"]).columns :
    df_espvie[col].fillna(df_espvie[col].median(), inplace=True)
#On enlève les dernières lignes qui sont inutiles
row_to_drop = df_espvie.loc[df_espvie["num_dep"]=="FE"].index[0]
df_espvie.drop(df_espvie.index[row_to_drop ::], inplace=True)
row_to_drop = df_espvie.loc[(df_espvie["num_dep"]=="P")|(df_espvie["num_dep"]=="M")].index
df_espvie.drop(row_to_drop, inplace=True)



IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# Nettoyage de la base des niveaux de vie

df_nvvie = df_nvvie.rename(columns= {"Départements": "num_dep", "Unnamed: 1" : "nom_dep"})
df_nvvie.dtypes
#On remplace les colonnes de type int ou float manquantes par la médiane
for col in df_nvvie.select_dtypes(["int64", "float64"]).columns :
    df_nvvie[col].fillna(df_nvvie[col].median(), inplace=True)
#Et on supprime les données pour toute la France métropolitaine et toute la France métropolitaine hors IdF
row_to_drop = df_nvvie.loc[df_nvvie["num_dep"]==974].index[0]
df_nvvie.drop(df_nvvie.index[row_to_drop+1 ::], inplace=True)
row_to_drop = df_nvvie.loc[(df_nvvie["num_dep"]=="P")|(df_nvvie["num_dep"]=="M")].index
df_nvvie.drop(row_to_drop, inplace=True)


In [4]:
df_nvvie

,num_dep,nom_dep,Rapport interdécile D9/D1
0,01,Ain,3.5
1,02,Aisne,3.1
2,03,Allier,3.0
3,04,Alpes-de-Haute-Provence,3.2
4,05,Hautes-Alpes,3.0
...,...,...,...
93,93,Seine-Saint-Denis,3.9
94,94,Val-de-Marne,4.2
95,95,Val-d'Oise,3.7
98,972,Martinique,4.2
